In [ ]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#%matplotlib inline
%matplotlib

### Using original non-atf data

In [ ]:
images = {
    "reference": {"filename":"../original_images/01.jpg"},
    "sterile_test": {"filename": "../original_images/02.jpg"},
    "real_world": {"filename": "../original_images/05.png"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Using new atf parts and whole images (matches to parts but not whole)

In [ ]:
images = {
    "reference": {"filename":"../atf_images/parts/3_butt.jpg"},
    "sterile_test": {"filename": "../atf_images/parts/6_butt.jpg"},
    "real_world": {"filename": "../atf_images/images/6.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Using other parts/whole atf images

In [ ]:
images = {
    "reference": {"filename":"../atf_images/parts/3_clip.jpg"},
    "sterile_test": {"filename": "../atf_images/parts/5_clip.jpg"},
    "real_world": {"filename": "../atf_images/images/5.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Pistol_22

In [ ]:
images = {
    "reference": {"filename":"../kris_images/Pistol_22/IMG_6082.jpg"},
    "sterile_test": {"filename": "../kris_images/Pistol_22/IMG_6107.jpg"},
    "real_world": {"filename": "../kris_images/Pistol_22/IMG_6079.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Winchester

In [ ]:
images = {
    "reference": {"filename":"../kris_images/Winchester/IMG_6123.jpg"},
    "sterile_test": {"filename": "../kris_images/Winchester/IMG_6123.jpg"},
    "real_world": {"filename": "../kris_images/Winchester/IMG_6122.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Winchester Small

In [ ]:
images = {
    "reference": {"filename":"../kris_images/Winchester_small/IMG_6121.JPG_resizeXpct.jpg"},
    "sterile_test": {"filename": "../kris_images/Winchester_small/IMG_6123.JPG_resizeXpct.jpg"},
    "real_world": {"filename": "../kris_images/Winchester_small/IMG_6122.JPG_resizeXpct.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Dragoon

In [ ]:
images = {
    "reference": {"filename":"../kris_images/Dragoon/IMG_6169.jpg"},
    "sterile_test": {"filename": "../kris_images/Dragoon/IMG_6169.jpg"},
    "real_world": {"filename": "../kris_images/Dragoon/IMG_6188.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

### Background removed

In [ ]:
images = {
    "reference": {"filename":"../kris_images/Pistol_22/IMG_6079_bgremoved.JPG"},
    "sterile_test": {"filename": "../kris_images/Pistol_22/IMG_6107.jpg"},
    "real_world": {"filename": "../kris_images/Pistol_22/IMG_6082.jpg"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

In [ ]:
fig, axes = plt.subplots(len(images))
fig.set_size_inches(6, 2.5*len(images))
for index, image in enumerate(["reference", "sterile_test", "real_world"]):
    axes[index].imshow(images[image]["image"])

In [ ]:
# detectors are for finding keypoints.  They often also support computing
detectors = {"fast": cv2.FastFeatureDetector_create(), # lots of points (2159) all over
             "brisk": cv2.BRISK_create(),              
             "akaze": cv2.AKAZE_create(),  # strange delocalization
             "kaze": cv2.KAZE_create(),  # strange delocalization
             "agast": cv2.AgastFeatureDetector_create(),
             "gftt": cv2.GFTTDetector_create(),
             "mser": cv2.MSER_create(),  # very few keypoints (80)
             "orb": cv2.ORB_create(),
             "star": cv2.xfeatures2d.StarDetector_create(),
             "sift": cv2.xfeatures2d.SIFT_create(),
             "surf": cv2.xfeatures2d.SURF_create(),
            }
descriptors_only = {
    "freak": cv2.xfeatures2d.FREAK_create(),
    "latch": cv2.xfeatures2d.LATCH_create(),
    "lucid": cv2.xfeatures2d.LUCID_create(1, 1),
}

In [ ]:
image = images["reference"]["image"]
for algorithm in detectors:
    plt.figure()
    kps = detectors[algorithm].detect(image)
    plt.imshow(cv2.drawKeypoints(image, kps, (255, 0,0)))
    plt.title(algorithm + " (%d pts)"%len(kps))

Compute features
================

In [ ]:
def compute_features(image, detector_alg, descriptor_alg=None):
    data = image["image"]
    if descriptor_alg in detectors:
        kps, descriptors = detectors[descriptor_alg].detectAndCompute(data, None)
    elif descriptor_alg in descriptors_only:
        kps = detectors[detector_alg].detect(data)
        kps, descriptors = descriptors_only[descriptor_alg].compute(data, kps)
    else:
        raise ValueError("unknown algorithm passed to descriptor stage")
    image["kps"] = kps
    image["descriptors"] = descriptors
    return image

Match keypoint descriptors
==========================

In [ ]:
def perspective_match(reference, unknown, use_flann=False, min_match_count=10,descriptor=None):
    if use_flann:
        FLANN_INDEX_KDTREE = 0
        FLANN_INDEX_LSH    = 6
        # floating point algorithms
        if descriptor in ["sift", "surf"]:
            index_params = dict(algorithm = FLANN_INDEX_KDTREE,
                                trees = 5)
        # binary algorithms
        else:
            index_params= dict(algorithm = FLANN_INDEX_LSH,
                                table_number = 6, # 12
                                key_size = 12,     # 20
                                multi_probe_level = 1) #2
        search_params = dict(checks = 50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
    else:
        matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(reference["descriptors"],
                               unknown["descriptors"],
                               k=2)
    good = []
    matchesMask=None
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    if len(good)>min_match_count:
        src_pts = np.float32([ reference["kps"][m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ unknown["kps"][m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        # this limits matches to being within the identified subimage
        try:
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
        except AttributeError:
            matchesMask, good = None, None

    else:
        print "Not enough matches are found (%d/%d)" % (len(good), min_match_count)
        matchesMask, good = None, None
    return matchesMask, good

In [ ]:
def draw_matches(reference_features, unknown_features, mask, good_pts):
    fig = plt.figure()
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = (255,0,0),
                       matchesMask = mask,
                       flags = 2)

    img3 = cv2.drawMatches(reference_features["image"],
                           reference_features["kps"],
                           unknown_features["image"],
                           unknown_features["kps"],
                           good_pts,None,**draw_params)
    plt.imshow(img3)
    return fig

In [ ]:
def wrapper(reference, unknown, detector_alg, 
            descriptor_alg=None, use_flann=False,
            min_match_count=5):
    if not descriptor_alg:
        descriptor_alg = detector_alg
    reference_features = compute_features(reference, 
                                          detector_alg, 
                                          descriptor_alg)
    unknown_features = compute_features(unknown, 
                                        detector_alg, 
                                        descriptor_alg)
    print('ref features: ',len(reference_features['kps']))
    print('unknown features: ',len(unknown_features['kps']))
    matchesMask, good_pts = perspective_match(reference_features,
                                              unknown_features,
                                             use_flann=use_flann,
                                             min_match_count=min_match_count,
                                             descriptor=descriptor_alg)
    fig = draw_matches(reference_features, unknown_features, 
                 matchesMask, good_pts)
    fig.gca().set_title("keypoints: {}, detector: {}, Matcher: {}".format(
        detector_alg, descriptor_alg, 
        "FLANN" if use_flann else "Brute Force"))
    if good_pts is not None:
        return len(good_pts)
    else:
        return 0

In [ ]:
def findAllCombs(a_list,b_list,ab_list):
    combs = []
    for a in a_list:
        for b in b_list:
            combs.append((a,b))
        for ab in ab_list:
            combs.append((a,ab))
    for b in b_list:
        for ab in ab_list:
            combs.append((ab,b))
    for ab in ab_list:
        combs.append((ab,ab))
    return combs

In [ ]:
both_list = ['brisk','akaze','kaze','orb']#,'sift','surf']
det_list = ['fast','agast','gftt','mser','star']
desc_list = ['freak','latch','lucid']

In [ ]:
df = pd.DataFrame(findAllCombs(det_list,desc_list,both_list),
                  columns=['detector','descriptor'])

In [ ]:
df['combo'] = df['detector']+df['descriptor']

In [ ]:
df = df.set_index('combo')

In [ ]:
for row in df.index:
    print(row)
    print(df.loc[row,'detector'])
    print(df.loc[row,'descriptor'])
    df.loc[row,'num_matches'] = wrapper(images["reference"], 
                                        images["real_world"], 
                                        detector_alg=df.loc[row,'detector'], 
                                        descriptor_alg=df.loc[row,'descriptor'])#,
                                        #use_flann=True)
    print(df.loc[row,'num_matches'])
    print('-'*40)

In [ ]:
df.sort_values('num_matches', ascending=False)

In [ ]:
df.sort_values('num_matches', ascending=False).to_csv("../performance_ranking_Winchester_small.csv")

In [ ]:
for i, comb in enumerate(findAllCombs(det_list,desc_list,both_list)):
#for i, comb in enumerate(test):
    if i < 8:
        print('-'*20)
        print('combo: '+str(i))
        print(comb)
        try:
            wrapper(images["reference"], images["real_world"], detector_alg=comb[0], descriptor_alg=comb[1])
        except AttributeError:
            print('Homography not found')
        except:
            print('there was an error')
        print('-'*20)

In [ ]:
wrapper(images["reference"], images["real_world"], "agast", descriptor_alg='latch')

### Homography Matching

In [ ]:
global coords1
coords1 = []

In [ ]:
ax = plt.gca()
fig = plt.gcf()
plt.title("Select 4 Points")

implot = ax.imshow(images["reference"]['image'])

def onclick(event):
    #global coords
    #coords = []
    if event.xdata != None and event.ydata != None:
        print(event.xdata, event.ydata)
        coords1.append((event.xdata, event.ydata))
    
        
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

In [ ]:
print coords1

In [ ]:
global coords2
coords2 = []

In [ ]:
ax = plt.gca()
fig = plt.gcf()
plt.title("Select 4 Points")

implot = ax.imshow(images["real_world"]['image'])

def onclick(event):
    #global coords
    #coords = []
    if event.xdata != None and event.ydata != None:
        print(event.xdata, event.ydata)
        coords2.append((event.xdata, event.ydata))
    
        
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

In [ ]:
print coords2

In [ ]:
h, status = cv2.findHomography(np.array(coords1),np.array(coords2))

In [ ]:
h

In [ ]:
plt.imshow(images["reference"]['image'])

In [ ]:
plt.imshow(images["real_world"]['image'])

In [ ]:
img_trans = cv2.warpPerspective(images["real_world"]['image'],h,
                                (images["reference"]['image'].shape[1],
                                 images["reference"]['image'].shape[0]))
plt.imshow(img_trans)